In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

import requests
import json
from datetime import datetime

# Prepair shapefile data
You need to prepair a shapefile of your the regions you want to download. It may have many columns, but the most important and must-appear is geometry column.

In [2]:
mekongReservoirs = gpd.read_file('mekongReservoirs')

In [3]:
startIndex = 70

In [4]:
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]

# GetModisData class
A class for easy download a subset of modis data. You only have to learn how to use the method:  
`GetModisData.createOrderURLURL(reservoirsGeoDataFrame, modisProductCode, email, recreateDownloadFile)`.

In [34]:
class GetModisData:
    url = "https://modis.ornl.gov/rst/api/v1/"
    header = {'Accept': 'application/json'} # Use following for a csv response: header = {'Accept': 'text/csv'}
    subsetUrl = 'https://modis.ornl.gov/subsetdata/'
    
    def calDistanceInKm(point1, point2):
        import pyproj
        geod = pyproj.Geod(ellps='WGS84')
        angle1, angle2, distance = geod.inv(point1.x, point1.y, point2.x, point2.y)
        return distance/1000

    
    def findBorderRect_1(polygon):
        from shapely.geometry import Point
        xmin, ymin, xmax, ymax = polygon.bounds

        topLeft = Point(xmin, ymax)
        bottomLeft = Point(xmin, ymin)
        topRight = Point(xmax, ymax)
        bottomRight = Point(xmax, ymin)
        vertices = [topLeft, topRight, bottomLeft, bottomRight]

        centroid = polygon.centroid
        kmAboveBelow = kmLeftRight \
                     = np.int32(np.ceil(max([GetModisData.calDistanceInKm(centroid, v) for v in vertices])))
        return centroid, kmAboveBelow, kmLeftRight
    
    
    def getMODISDate(reservoirRowDf):
        # Submit request
        row = reservoirRowDf
        response = requests.get(GetModisData.url
                                + row['product'] 
                                + '/dates?latitude=' + str(row['latitude']) 
                                + '&longitude='+ str(row['longitude']), headers=GetModisData.header)

        # Get dates object as list of python dictionaries
        dates = json.loads(response.text)['dates'] 

        # Convert to list of tuples; change calendar_date key values to datetimes
        dates = [(datetime.strptime(date['calendar_date'], "%Y-%m-%d"), date['modis_date']) for date in dates]

        # Get MODIS dates nearest to start_date and end_date and add to new pandas columns
        start_MODIS_date = min(date[1] for date in dates if date[0] > row['start_date'])
        end_MODIS_date = max(date[1] for date in dates if date[0] < row['end_date'])

        return start_MODIS_date, end_MODIS_date
    
    
    def createReservoirsDownloadDataframe(reservoirsGeoDataFrame, productCode, email, startIndex):
        df = pd.DataFrame(columns=['site_id', 'product', 'latitude', 'longitude', 
                                   'email', 'start_date', 'end_date',
                                   'kmAboveBelow', 'kmLeftRight', 'order_uid'])
        
        default_start_date = pd.to_datetime('2000-01-01')
        default_end_date = pd.to_datetime('2018-07-25')
        
        for index, reservoirRow in reservoirsGeoDataFrame.iterrows():
            center, kmAboveBelow, kmLeftRight = GetModisData.findBorderRect_1(reservoirRow.geometry)
            row = {'site_id': 'site{}'.format(index),
                    'product': productCode,
                    'latitude': center.y,
                    'longitude': center.x,
                    'email': email,
                    'start_date': default_start_date,
                    'end_date': default_end_date,
                    'kmAboveBelow': kmAboveBelow,
                    'kmLeftRight': kmLeftRight,
                    'order_uid': 'None'}
            
            row['start_date'], row['end_date'] = GetModisData.getMODISDate(row)
            df = df.append(row, ignore_index=True)
        
        df.index = pd.RangeIndex(startIndex, startIndex + df.shape[0])
        return df
    
    
    def createOrderURL_1(reservoirsDownloadDf, downloadFileResult=None):
        # Make list to collect order UIDs
        order_uids = []
        
        for index, row in reservoirsDownloadDf.iterrows():
            if row['order_uid'] == 'None':
                # Build request URL
                requestURL = GetModisData.url + row['product'] \
                                              + "/subsetOrder?latitude=" + str(row['latitude']) \
                                              + "&longitude=" + str(row['longitude']) \
                                              + "&email=" + row['email'] \
                                              + "&uid=" + row['site_id'] \
                                              + "&startDate=" + row['start_date'] \
                                              + "&endDate=" + row['end_date'] \
                                              + "&kmAboveBelow=" + str(row['kmAboveBelow']) \
                                              + "&kmLeftRight=" + str(row['kmLeftRight'])

                print(requestURL)
                # Submit request
                response = requests.get(requestURL, headers=GetModisData.header)
                try:
                    order_uid = GetModisData.subsetUrl + json.loads(response.text)['order_id']
                except TypeError as err:
                    print(response.text)
                    break

                # Append UID to list
                order_uids.append(order_uid)
                reservoirsDownloadDf.loc[index, 'order_uid'] = order_uid

        print(order_uids)
        if downloadFileResult is None:
            reservoirsDownloadDf.to_csv(productCode + '.csv')
        else:
            reservoirsDownloadDf.to_csv(downloadFileResult + '.csv')
        return order_uids
    
    
    # Main function to get MODIS data
    def createOrderURL(reservoirsGeoDataFrame, modisProductCode='MOD13Q1', 
                       email='lamductan@gmail.com', recreateDownloadFile=False,
                       downloadFileResult=None, startIndex=0):
        import os
        print('Start')
        if downloadFileResult is None:
            downloadFileResult = modisProductCode
        if not os.path.exists(downloadFileResult + '.csv') or recreateDownloadFile:
            print('1')
            with open(downloadFileResult + '.csv', 'a') as f:
                print('2')
                #reservoirsDownloadDf = pd.DataFrame(columns=['site_id', 'product', 'latitude', 'longitude', 
                #                       'email', 'start_date', 'end_date',
                #                       'kmAboveBelow', 'kmLeftRight', 'order_uid'])
                emailId = 3
                for idx in np.arange(startIndex, 500, 20):
                    print('idx = ', idx)
                    email = '1616' + str(emailId).zfill(3) + '@student.hcmus.edu.vn'
                    print('email = ', email)
                    downloadGeoDataFrame = reservoirsGeoDataFrame.iloc[idx-startIndex: idx-startIndex+20]
                    reservoirsDownloadDf = GetModisData.createReservoirsDownloadDataframe(downloadGeoDataFrame, 
                                                                                    modisProductCode, email, idx)
                    reservoirsDownloadDf.to_csv(f, header=(idx==startIndex))
                    emailId += 1
        else:
            reservoirsDownloadDf = pd.read_csv(downloadFileResult + '.csv')
        return GetModisData.createOrderURL_1(reservoirsDownloadDf, downloadFileResult)

# Example how to use GetModisData class

In [6]:
startIndex = 90
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]

downloadTerraUrlList_0 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='lamductan@outlook.com',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/0_MOD13Q1',
                                                  startIndex=startIndex)
print()
startIndex += 20
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]
downloadTerraUrlList_1 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='1616095@student.hcmus.edu.vn',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/1_MOD13Q1',
                                                  startIndex=startIndex)

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.728625699383002&longitude=104.17371104780501&email=lamductan@outlook.com&uid=site90&startDate=A2000049&endDate=A2018193&kmAboveBelow=3&kmLeftRight=3
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.722557330060546&longitude=102.85882501376895&email=lamductan@outlook.com&uid=site91&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.707732737823086&longitude=104.2096375725413&email=lamductan@outlook.com&uid=site92&startDate=A2000049&endDate=A2018193&kmAboveBelow=3&kmLeftRight=3
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.68867626753318&longitude=104.29890657379126&email=lamductan@outlook.com&uid=site93&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.68843798715557&longitude=104.03700010171463&email=lamductan@outlook.com&uid=

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.161472284265074&longitude=104.31068846324419&email=1616095@student.hcmus.edu.vn&uid=site118&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.150533187668515&longitude=104.60149614989275&email=1616095@student.hcmus.edu.vn&uid=site119&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.11931501449505&longitude=104.19286879773699&email=1616095@student.hcmus.edu.vn&uid=site120&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.102939409537292&longitude=102.9353300774543&email=1616095@student.hcmus.edu.vn&uid=site121&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=17.046556661198807&longitude=103.04323792737003

In [7]:
startIndex += 20
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]

downloadTerraUrlList_2 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='lamductan@gmail.com',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/2_MOD13Q1',
                                                  startIndex=startIndex)
print()
startIndex += 20
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]
downloadTerraUrlList_3 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='1512489@student.hcmus.edu.vn',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/3_MOD13Q1',
                                                  startIndex=startIndex)

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.77161299605363&longitude=102.76814123356083&email=lamductan@gmail.com&uid=site130&startDate=A2000049&endDate=A2018193&kmAboveBelow=3&kmLeftRight=3
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.731483759276415&longitude=102.12917425894965&email=lamductan@gmail.com&uid=site131&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.70562593894028&longitude=103.86104015232503&email=lamductan@gmail.com&uid=site132&startDate=A2000049&endDate=A2018193&kmAboveBelow=3&kmLeftRight=3
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.686767632856174&longitude=104.18782114497533&email=lamductan@gmail.com&uid=site133&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.678905899879474&longitude=103.89274167229243&email=lamductan@gmail.com&uid=site1

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.039522835489713&longitude=103.27174744710945&email=1512489@student.hcmus.edu.vn&uid=site158&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.04731459218253&longitude=102.04938275869512&email=1512489@student.hcmus.edu.vn&uid=site159&startDate=A2000049&endDate=A2018193&kmAboveBelow=3&kmLeftRight=3
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.024597224724157&longitude=104.0240575223352&email=1512489@student.hcmus.edu.vn&uid=site160&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.96964346649513&longitude=102.35762670312646&email=1512489@student.hcmus.edu.vn&uid=site161&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.97001501290234&longitude=104.0053245344735&em

In [8]:
startIndex += 20
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]

downloadTerraUrlList_4 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='leduongtuananh97@gmail.com',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/4_MOD13Q1',
                                                  startIndex=startIndex)
print()
startIndex += 20
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]
downloadTerraUrlList_5 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='1512002@student.hcmus.edu.vn',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/5_MOD13Q1',
                                                  startIndex=startIndex)

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.788845753815627&longitude=102.33531122363814&email=leduongtuananh97@gmail.com&uid=site170&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.794263515848105&longitude=102.59846672593572&email=leduongtuananh97@gmail.com&uid=site171&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.751377697830439&longitude=104.3478153117737&email=leduongtuananh97@gmail.com&uid=site172&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.739676287057002&longitude=104.37375187903318&email=leduongtuananh97@gmail.com&uid=site173&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.723502794451056&longitude=102.21936184495303&email=

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.808542362214544&longitude=103.31885503212912&email=1512002@student.hcmus.edu.vn&uid=site197&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.75114292900308&longitude=103.56137267720123&email=1512002@student.hcmus.edu.vn&uid=site198&startDate=A2000049&endDate=A2018193&kmAboveBelow=4&kmLeftRight=4
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.715836171955084&longitude=104.58388475181533&email=1512002@student.hcmus.edu.vn&uid=site199&startDate=A2000049&endDate=A2018193&kmAboveBelow=4&kmLeftRight=4
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.666723838657648&longitude=101.85659799828683&email=1512002@student.hcmus.edu.vn&uid=site200&startDate=A2000049&endDate=A2018193&kmAboveBelow=4&kmLeftRight=4
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.665104764423138&longitude=102.7885167776227

In [9]:
startIndex += 20
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]

downloadTerraUrlList_6 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='1616001@student.hcmus.edu.vn',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/6_MOD13Q1',
                                                  startIndex=startIndex)
print()
startIndex += 20
downloadReservoirs = mekongReservoirs.iloc[startIndex:startIndex + 20]
downloadTerraUrlList_7 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='1616002@student.hcmus.edu.vn',
                                                  recreateDownloadFile=True,
                                                  downloadFileResult='DownloadFiles/7_MOD13Q1',
                                                  startIndex=startIndex)

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.487842939238918&longitude=104.57597252685706&email=1616001@student.hcmus.edu.vn&uid=site210&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.451642583674323&longitude=103.61195706619024&email=1616001@student.hcmus.edu.vn&uid=site211&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.450038607056811&longitude=105.04485197709481&email=1616001@student.hcmus.edu.vn&uid=site212&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.431589176710393&longitude=104.83465054711392&email=1616001@student.hcmus.edu.vn&uid=site213&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=14.427388020271415&longitude=103.790550764243

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.807324797522403&longitude=104.84972651307643&email=1616002@student.hcmus.edu.vn&uid=site237&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.794834550451924&longitude=108.44618530836338&email=1616002@student.hcmus.edu.vn&uid=site238&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.747034119250966&longitude=108.44805523519918&email=1616002@student.hcmus.edu.vn&uid=site239&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.744063736178383&longitude=108.19804196115317&email=1616002@student.hcmus.edu.vn&uid=site240&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.728879442131223&longitude=104.675161139996

In [14]:
print(startIndex)

70


In [16]:
print('next startIndex = ', startIndex + 20)

next startIndex =  90


In [35]:
startIndex = 250
downloadReservoirs = mekongReservoirs.iloc[startIndex:500]
downloadTerraUrlList_250_500 = GetModisData.createOrderURL(reservoirsGeoDataFrame=downloadReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  recreateDownloadFile=False,
                                                  downloadFileResult='DownloadFiles/250-500_MOD13Q1',
                                                  startIndex=startIndex)

Start
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.533368697292492&longitude=106.01036036988572&email=1616003@student.hcmus.edu.vn&uid=site250&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.489405508713435&longitude=104.98478931311035&email=1616003@student.hcmus.edu.vn&uid=site251&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.47564354588265&longitude=106.0318773703182&email=1616003@student.hcmus.edu.vn&uid=site252&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.43411037471098&longitude=104.9863415216738&email=1616003@student.hcmus.edu.vn&uid=site253&startDate=A2000049&endDate=A2018193&kmAboveBelow=5&kmLeftRight=5
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.441193299044848&longitude=107.6099921437

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.993688884828394&longitude=104.77450333721308&email=1616004@student.hcmus.edu.vn&uid=site287&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.995949299529725&longitude=105.10133976225995&email=1616004@student.hcmus.edu.vn&uid=site288&startDate=A2000049&endDate=A2018193&kmAboveBelow=3&kmLeftRight=3
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.97082272262021&longitude=104.90569984315586&email=1616004@student.hcmus.edu.vn&uid=site289&startDate=A2000049&endDate=A2018193&kmAboveBelow=4&kmLeftRight=4
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.971604561262918&longitude=105.36727389125015&email=1616005@student.hcmus.edu.vn&uid=site290&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.976402357564963&longitude=105.5264131358168

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.089855047866402&longitude=104.98585461102199&email=1616006@student.hcmus.edu.vn&uid=site324&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.0702663473227&longitude=104.79398747844364&email=1616006@student.hcmus.edu.vn&uid=site325&startDate=A2000049&endDate=A2018193&kmAboveBelow=3&kmLeftRight=3
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.049907189418551&longitude=104.77829352307025&email=1616006@student.hcmus.edu.vn&uid=site326&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.056592530939513&longitude=105.29936745373764&email=1616006@student.hcmus.edu.vn&uid=site327&startDate=A2000049&endDate=A2018193&kmAboveBelow=4&kmLeftRight=4
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=11.004422310405634&longitude=104.7709994017502&

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=23.49983710307397&longitude=99.99122781019476&email=1616008@student.hcmus.edu.vn&uid=site361&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=23.489562248875824&longitude=100.31939277235377&email=1616008@student.hcmus.edu.vn&uid=site362&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=23.458532901665023&longitude=99.97469398573458&email=1616008@student.hcmus.edu.vn&uid=site363&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=23.43115267472759&longitude=100.51760525100329&email=1616008@student.hcmus.edu.vn&uid=site364&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=23.405304988380216&longitude=99.92555148056131&em

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=20.22770815516788&longitude=100.3602286631934&email=1616010@student.hcmus.edu.vn&uid=site398&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=20.21674268338275&longitude=99.9543544646051&email=1616010@student.hcmus.edu.vn&uid=site399&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=20.206573347009453&longitude=99.94853215737713&email=1616010@student.hcmus.edu.vn&uid=site400&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=20.192937452136626&longitude=100.02405803595232&email=1616010@student.hcmus.edu.vn&uid=site401&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=20.1637638789572&longitude=99.79612203441592&email=

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=19.375716426440825&longitude=99.88882835304494&email=1616012@student.hcmus.edu.vn&uid=site435&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=19.360321349037324&longitude=99.82168711248998&email=1616012@student.hcmus.edu.vn&uid=site436&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=19.333502196912626&longitude=100.20434757147802&email=1616012@student.hcmus.edu.vn&uid=site437&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=19.332910449191235&longitude=99.96918270188507&email=1616012@student.hcmus.edu.vn&uid=site438&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=19.330633763545343&longitude=99.9393050025416&em

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=18.33012745431079&longitude=103.22505202252573&email=1616014@student.hcmus.edu.vn&uid=site472&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=18.318387887851863&longitude=103.30692893250222&email=1616014@student.hcmus.edu.vn&uid=site473&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=18.304153459472133&longitude=103.82920455802899&email=1616014@student.hcmus.edu.vn&uid=site474&startDate=A2000049&endDate=A2018193&kmAboveBelow=2&kmLeftRight=2
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=18.269075847526924&longitude=103.72520152946352&email=1616014@student.hcmus.edu.vn&uid=site475&startDate=A2000049&endDate=A2018193&kmAboveBelow=1&kmLeftRight=1
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=18.26896848803197&longitude=102.770257480365&e